#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [1]:
pca_variance = 0.94    
roi_crop_size = (256,256)
patch_size = (128,128)
pixels_per_cell=(32,32)
cells_per_block=(3,3)
orientations=9

In [2]:
import os, re, joblib
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, greycomatrix, greycoprops
from skimage.filters import threshold_otsu, median
from skimage.morphology import opening, disk
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from joblib import Parallel, delayed
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
def process_single_image(args):
    img_path, label, classes = args
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, roi_crop_size)
    img = img.astype(np.float32)

    center = np.array(roi_crop_size) // 2
    ps = np.array(patch_size) // 2
    patch = img[center[0]-ps[0]:center[0]+ps[0], center[1]-ps[1]+0:center[1]+ps[1]]

    patch = median(patch)
    patch = cv2.GaussianBlur(patch, (3, 3), 0)
    patch = patch.astype(np.float32) / 255.0

    return patch, classes.index(label)


In [4]:

def extract_and_preprocess_parallel(root_dir, classes, n_jobs=-1):
    pattern = re.compile(r'^[A-Z]_\d+_\d+\.(LEFT|RIGHT)_(CC|MLO)\.jpg$', re.IGNORECASE)
    siniflar = ['benign', 'cancer', 'normal']
    yollar, etiketler = [], []

    for cls in siniflar:
        cls_yolu = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_yolu):
            continue
        for durum_id in os.listdir(cls_yolu):
            durum_yolu = os.path.join(cls_yolu, durum_id)
            if not os.path.isdir(durum_yolu):
                continue
            for dosya_adi in os.listdir(durum_yolu):
                if pattern.match(dosya_adi):
                    yollar.append(os.path.join(durum_yolu, dosya_adi))
                    etiketler.append(cls)

    # === Paralel işleme ===
    arg_list = [(img_path, lbl, classes) for img_path, lbl in zip(yollar, etiketler)]
    results = Parallel(n_jobs=n_jobs)(delayed(process_single_image)(args) for args in arg_list)

    X, y = zip(*results)
    return np.array(X), np.array(y)

In [5]:
def extract_features_from_image(im):
    h = hog(im, pixels_per_cell=pixels_per_cell,
                cells_per_block=cells_per_block,
                orientations=orientations)
    g = greycomatrix((im*255).astype(np.uint8), [1], [0], levels=256)
    glcm = [greycoprops(g, prop)[0, 0] for prop in ('contrast', 'homogeneity', 'energy')]
    area = np.sum(im > 0.5)
    perimeter = np.sum(cv2.Canny((im * 255).astype(np.uint8), 50, 150) > 0)
    ecc = 0
    if area > 10:
        pts = np.argwhere(im > 0.5)
        rect = cv2.minAreaRect(pts.astype(np.float32))
        ecc = rect[1][0] / (rect[1][1] + 1e-8)
    return np.hstack([h, glcm, area, perimeter, ecc])

def extract_features(X, out_prefix=None, n_jobs=-1):
    # 2. UPDATE THE FUNCTION NAME INSIDE THE PARALLEL CALL HERE
    feats = Parallel(n_jobs=n_jobs)(delayed(extract_features_from_image)(im) for im in X)
    feats = np.array(feats)
    pca = PCA(n_components=pca_variance)
    X_feat = pca.fit_transform(feats)
    return X_feat, pca

 Metirkler**

In [6]:
if __name__ == '__main__':
    classes = ['normal','benign','cancer']
    kok_dizin = os.path.join(os.getcwd(), "Project1")
    X_test, y_test = extract_and_preprocess_parallel(kok_dizin, classes,n_jobs=-1)
    X_feat, pca = extract_features(X_test)
    model = load_model("MLP_1_1.h5")
    T_m = 0.40
    T_b = 0.40
    probs = model.predict(X_feat)
    preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
    print("\n --------------MLP RESULTS----------------")
    print(classification_report(y_test, preds, target_names=classes))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Precision:", precision_score(y_test, preds, average='macro'))
    print("Recall:", recall_score(y_test, preds, average='macro'))
    print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
    try:
        print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
    except:
        print("AUC hesaplanamadı.")
    


 --------------MLP RESULTS----------------
              precision    recall  f1-score   support

      normal       0.33      1.00      0.50        80
      benign       0.00      0.00      0.00        80
      cancer       0.00      0.00      0.00        80

    accuracy                           0.33       240
   macro avg       0.11      0.33      0.17       240
weighted avg       0.11      0.33      0.17       240

Confusion Matrix:
 [[80  0  0]
 [80  0  0]
 [80  0  0]]
Accuracy: 0.3333333333333333
Precision: 0.1111111111111111
Recall: 0.3333333333333333
📌 F1 Score: 0.16666666666666666
AUC: 0.5020833333333333


c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Dr.Ahmet\anaconda3\envs\tf-gpu-211\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [7]:
model = joblib.load("RF_1_1.pkl")
T_m = 0.40
T_b = 0.40
classes = ['normal', 'benign', 'cancer']
label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
probs = model.predict_proba(X_feat) 
preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
print("\n --------------RF RESULTS----------------")
print(classification_report(y_test, preds, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds, average='macro'))
print("Recall:", recall_score(y_test, preds, average='macro'))
print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
except:
    print("AUC hesaplanamadı.")


 --------------RF RESULTS----------------
              precision    recall  f1-score   support

      normal       0.38      0.36      0.37        80
      benign       0.36      0.45      0.40        80
      cancer       0.38      0.30      0.33        80

    accuracy                           0.37       240
   macro avg       0.37      0.37      0.37       240
weighted avg       0.37      0.37      0.37       240

Confusion Matrix:
 [[29 32 19]
 [23 36 21]
 [24 32 24]]
Accuracy: 0.37083333333333335
Precision: 0.37219298245614035
Recall: 0.37083333333333335
📌 F1 Score: 0.36837606837606834
AUC: 0.5394270833333333


In [8]:
loaded_xgb_model_booster = xgb.Booster() 
loaded_xgb_model_booster.load_model("XGBoost_1_1.json") 
T_m = 0.40
T_b = 0.40
classes = ['normal', 'benign', 'cancer']
label_map = {'normal': 0, 'benign': 1, 'cancer': 2}


dtest_predict = xgb.DMatrix(X_feat) 
probs = loaded_xgb_model_booster.predict(dtest_predict) 
preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
T_m = 0.40
T_b = 0.40

print("\n --------------XP-B RESULTS----------------")
print(classification_report(y_test, preds, target_names=classes))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:", precision_score(y_test, preds, average='macro'))
print("Recall:", recall_score(y_test, preds, average='macro'))
print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
except:
    print("AUC hesaplanamadı.")


 --------------XP-B RESULTS----------------
              precision    recall  f1-score   support

      normal       0.37      0.49      0.42        80
      benign       0.38      0.14      0.20        80
      cancer       0.36      0.47      0.41        80

    accuracy                           0.37       240
   macro avg       0.37      0.37      0.34       240
weighted avg       0.37      0.37      0.34       240

Confusion Matrix:
 [[39  8 33]
 [34 11 35]
 [32 10 38]]
Accuracy: 0.36666666666666664
Precision: 0.36974316076463115
Recall: 0.3666666666666667
📌 F1 Score: 0.34401954484819236
AUC: 0.5936718750000001
